In [1]:
import pandas as pd
import pandas_profiling
from pandas.io.json import json_normalize

## Products data set 

In [2]:
df_products = pd.read_json("data/products.json.gz")

In [3]:
df_products.head(3)

,category,description,image,manufacturer,model,name,price,shipping,sku,type,upc,url
0,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",Compatible with select electronic devices; AAA...,http://img.bbystatic.com/BestBuy_US/images/pro...,Duracell,MN2400B4Z,Duracell - AAA Batteries (4-Pack),5.49,5.49,43900,HardGood,41333424019,http://www.bestbuy.com/site/duracell-aaa-batte...
1,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",Long-lasting energy; DURALOCK Power Preserve t...,http://img.bbystatic.com/BestBuy_US/images/pro...,Duracell,MN1500B4Z,Duracell - AA 1.5V CopperTop Batteries (4-Pack),5.49,5.49,48530,HardGood,41333415017,http://www.bestbuy.com/site/duracell-aa-1-5v-c...
2,"[{'id': 'pcmcat312300050015', 'name': 'Connect...",Compatible with select electronic devices; AA ...,http://img.bbystatic.com/BestBuy_US/images/pro...,Duracell,MN1500B8Z,Duracell - AA Batteries (8-Pack),7.49,5.49,127687,HardGood,41333825014,http://www.bestbuy.com/site/duracell-aa-batter...


#### Desciption of attributes
* image: url to product image
* upc: Universal Product Code (UPC)
* sku: Best Buy unique 7-digit product identifier

##### Links/references: 
* https://bestbuyapis.github.io/api-documentation/#detail
* https://bestbuyapis.github.io/bby-query-builder/#/productSearch

#### Processing "category" column
* As the **category** field consists of a list of jsons, we will process the column and create multiple columns for each category. 
* Why are we doing this? If you would like to predict a category you will need to use this feature...

In [4]:
# Processing category field with json_normalize [http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.io.json.json_normalize.html]
df_list = []
for x in range(len(df_products)):
    df_temp = json_normalize(df_products["category"][x])
    df_temp = df_temp.transpose()
    df_temp = df_temp.drop(["name"])
    df_temp = df_temp.rename(index={"id": x})
    df_list.append(df_temp)
df_category = pd.concat(df_list)
df_products = df_products.join(df_category)
# Delete temp dataframes..
del df_list, df_temp, df_category
# Renaming category column to respective fields
df_products.rename(index=str, columns={0: "Category_F1", 1: "Category_F2", 2: "Category_F3", 3: "Category_F4", 4: "Category_F5", 5: "Category_F6", 6: "Category_F7"}, inplace=True)
# Dropping columns that dont provide substantial information for analysis of data
df_products.drop(['sku', 'upc','url', 'image','category'], axis=1, inplace=True)

In [5]:
df_products.head(3)

,description,manufacturer,model,name,price,shipping,type,Category_F1,Category_F2,Category_F3,Category_F4,Category_F5,Category_F6,Category_F7
0,Compatible with select electronic devices; AAA...,Duracell,MN2400B4Z,Duracell - AAA Batteries (4-Pack),5.49,5.49,HardGood,pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,abcat0208002,NaN,NaN,NaN
1,Long-lasting energy; DURALOCK Power Preserve t...,Duracell,MN1500B4Z,Duracell - AA 1.5V CopperTop Batteries (4-Pack),5.49,5.49,HardGood,pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,abcat0208002,NaN,NaN,NaN
2,Compatible with select electronic devices; AA ...,Duracell,MN1500B8Z,Duracell - AA Batteries (8-Pack),7.49,5.49,HardGood,pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,abcat0208002,NaN,NaN,NaN


## Analysis of data
* Tool used for explanation is pandas-profiling
    * Link: https://github.com/pandas-profiling/pandas-profiling
* Some features of pandas-profiling:
    * Generates a neat html report of a dataframe
    * User can interactively explore attributes

In [6]:
pandas_profiling.ProfileReport(df_products)

Number of variables,15
Number of observations,51646
Total Missing (%),23.8%
Total size in memory,5.9 MiB
Average record size in memory,120.0 B
Numeric,1
Categorical,13
Boolean,0
Date,0
Text (Unique),1
Rejected,0
